In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import csv # implement classes to read and write tabular data in CSV form

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [46]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from PIL import Image # converting images into arrays
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

# install wordcloud
!conda install -c conda-forge wordcloud==1.4.1 --yes

# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS

print ('Wordcloud is installed and imported!')

Matplotlib version:  3.0.2
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - wordcloud==1.4.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wordcloud-1.4.1            |           py36_0         324 KB  conda-forge

The following NEW packages will be INSTALLED:

    wordcloud: 1.4.1-py36_0 conda-forge


wordcloud-1.4.1      | 324 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Wordcloud is installed and imported!


In [84]:
CLIENT_ID = 'VJQGAVEXHBNZEZH21GCRYSAAAZXTFJXW2LDAMFH1VQBLFRGN' # your Foursquare ID
CLIENT_SECRET = 'ZL3Y20OR4ERAW02R22WYW0FPH32G5T5WH0LTS1NT1BHJJOJA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

My Credentails:
CLIENT_ID: VJQGAVEXHBNZEZH21GCRYSAAAZXTFJXW2LDAMFH1VQBLFRGN


In [88]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Houston, TX']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [89]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [90]:
df_venues

{'New York, NY':                                    Name                      Address  \
 0                                B Side                370 W 51st St   
 1                   Prince Street Pizza                 27 Prince St   
 2                           Joe's Pizza                150 E 14th St   
 3                    Emily West Village                35 Downing St   
 4                           Joe's Pizza                 7 Carmine St   
 5                                 Posto                  310 2nd Ave   
 6         San Matteo Pizza Espresso Bar                 1739 2nd Ave   
 7                        Champion Pizza              17 Cleveland Pl   
 8                         Scarr's Pizza                22 Orchard St   
 9                           Joe's Pizza                1435 Broadway   
 10            John's of Bleecker Street              278 Bleecker St   
 11            Garlic New York Pizza Bar                  629 2nd Ave   
 12               Vezzo Thin Crust 

In [91]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    
    print("Showing Top 100")
    print("_________________________________________________________")

Total number of pizza places in New York, NY =  271
Showing Top 100
_________________________________________________________
Total number of pizza places in Houston, TX =  191
Showing Top 100
_________________________________________________________


In [92]:
maps[cities[0]]

In [93]:
maps[cities[1]]

In [94]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
       
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))
    print("___________________________________")

New York, NY
Mean Distance from Mean coordinates
0.02296898464975829
___________________________________
Houston, TX
Mean Distance from Mean coordinates
0.1289897996088065
___________________________________


In [95]:
maps[cities[0]]

In [96]:
maps[cities[1]]

In [97]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Houston, TX']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1df931735") # BBQ CATEGORY ID
    results[city] = requests.get(url).json()

In [98]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [99]:
df_venues

{'New York, NY':                                       Name  \
 0                       Mighty Quinn's BBQ   
 1                       Mighty Quinn's BBQ   
 2   Baekjeong NYC (Kang Ho Dong Baekjeong)   
 3             Hill Country Barbecue Market   
 4                               Jongro BBQ   
 5                    Gyu-Kaku Japanese BBQ   
 6                    Gyu-Kaku Japanese BBQ   
 7                               Blue Smoke   
 8                               Blue Smoke   
 9                           miss KOREA BBQ   
 10           Don's Bogam Korean BBQ & Wine   
 11                       Virgil's Real BBQ   
 12                             Korilla BBQ   
 13                      Mighty Quinn's BBQ   
 14                      Mighty Quinn's BBQ   
 15                             Strip House   
 16                       Samwon Garden BBQ   
 17                      Mighty Quinn's BBQ   
 18                 Churrascaria Plataforma   
 19                               New Wonjo 

In [100]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of BBQ places in {city} = ", results[city]['response']['totalResults'])
    
    print("Showing Top 100")
    print("_________________________________________________________")

Total number of BBQ places in New York, NY =  206
Showing Top 100
_________________________________________________________
Total number of BBQ places in Houston, TX =  235
Showing Top 100
_________________________________________________________


In [101]:
maps[cities[0]]

In [102]:
maps[cities[1]]

In [103]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
       
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))
    print("___________________________________")

New York, NY
Mean Distance from Mean coordinates
0.032092500224089264
___________________________________
Houston, TX
Mean Distance from Mean coordinates
0.1406213779246011
___________________________________


In [104]:
maps[cities[0]]

In [105]:
maps[cities[1]]

In [106]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Houston, TX']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d155941735") # GASTROPUB CATEGORY ID
    results[city] = requests.get(url).json()

In [107]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [108]:
df_venues

{'New York, NY':                                         Name               Address        Lat  \
 0                                The Penrose          1590 2nd Ave  40.775444   
 1                                Bar Sardine         183 W 10th St  40.734334   
 2                                   The Wren            344 Bowery  40.726322   
 3              The Breslin Bar & Dining Room          16 W 29th St  40.745810   
 4                                      Ousia         629 w 57th st  40.770905   
 5                                  As Is NYC          734 10th Ave  40.764685   
 6                              Wilfie & Nell          228 W 4th St  40.734064   
 7                    OSAMIL Korean Gastropub           5 W 31st St  40.746864   
 8                   Hudson & Charles Dinette         522 Hudson St  40.733936   
 9                              Mother's Ruin          18 Spring St  40.721311   
 10                          Good Night Sonny           134 1st Ave  40.727598   


In [109]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Gastropub places in {city} = ", results[city]['response']['totalResults'])
    
    print("Showing Top 100")
    print("_________________________________________________________")

Total number of Gastropub places in New York, NY =  129
Showing Top 100
_________________________________________________________
Total number of Gastropub places in Houston, TX =  61
Showing Top 100
_________________________________________________________


In [110]:
maps[cities[0]]

In [111]:
maps[cities[1]]

In [112]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
       
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))
    print("___________________________________")

New York, NY
Mean Distance from Mean coordinates
0.023997938650522882
___________________________________
Houston, TX
Mean Distance from Mean coordinates
0.10912528626798963
___________________________________


In [113]:
maps[cities[0]]

In [114]:
maps[cities[1]]

In [115]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Houston, TX']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1c1941735") # MEXICAN RESTAURANT  CATEGORY ID
    results[city] = requests.get(url).json()

In [116]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [117]:
df_venues

{'New York, NY':                                       Name  \
 0                               La Esquina   
 1                             Barrio Chino   
 2                  Tacombi at Fonda Nolita   
 3                       Taqueria St. Marks   
 4                              La Contenta   
 5                                 Toloache   
 6            Zaragoza Mexican Deli-Grocery   
 7                                    Fonda   
 8                                  Ofrenda   
 9                              Toloache 82   
 10                           Rosa Mexicano   
 11                                   Cosme   
 12                 JaJaJa Plantas Mexicana   
 13                         Downtown Bakery   
 14              Tacombi Café El Presidente   
 15                           Arriba Arriba   
 16                                 Rosie's   
 17                              El Parador   
 18                           La Newyorkina   
 19                           Rosa Mexicano 

In [118]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Mexican Restuarants in {city} = ", results[city]['response']['totalResults'])
    
    print("Showing Top 100")
    print("_________________________________________________________")

Total number of Mexican Restuarants in New York, NY =  293
Showing Top 100
_________________________________________________________
Total number of Mexican Restuarants in Houston, TX =  316
Showing Top 100
_________________________________________________________


In [119]:
maps[cities[0]]

In [120]:
maps[cities[1]]

In [121]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
       
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))
    print("___________________________________")

New York, NY
Mean Distance from Mean coordinates
0.025767126694086838
___________________________________
Houston, TX
Mean Distance from Mean coordinates
0.12310117617757761
___________________________________


In [122]:
maps[cities[0]]

In [123]:
maps[cities[1]]